In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
import logging
import numpy as np

# Configuración de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configuración del modelo
MODEL_NAME = "bert-base-multilingual-cased"
OUTPUT_DIR = "./models/bert_news_classifier"
NUM_LABELS = 5  # Número de etiquetas para la clasificación

In [ ]:


# Cargar el dataset
dataset = pd.read_csv("dataset_unificado_preprocesado.csv")

# Dividir datos en entrenamiento y validación
texts = dataset["Contenido"] + " " + dataset["Título"]  # Combina contenido y título
labels = dataset["etiqueta"]  # Asegúrate de que las etiquetas sean numéricas (1-5)

X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Cargar el tokenizador y tokenizar los textos
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512, return_tensors="pt")
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Crear dataset compatible con PyTorch
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# Crear datasets de entrenamiento y validación
train_dataset = NewsDataset(train_encodings, y_train.tolist())
val_dataset = NewsDataset(val_encodings, y_val.tolist())

# Cargar modelo
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_LABELS)

# Configuración de argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,               # Directorio para checkpoints
    eval_strategy="steps",              # Evaluar durante el entrenamiento
    save_strategy="steps",              # Guardar checkpoints periódicamente
    save_steps=500,                      # Guardar cada 500 pasos
    save_total_limit=3,                  # Mantener solo los 3 últimos checkpoints
    num_train_epochs=5,                  # Número de épocas
    per_device_train_batch_size=8,       # Tamaño de batch por dispositivo
    per_device_eval_batch_size=8,        # Tamaño de batch para evaluación
    logging_dir="./logs",               # Directorio para logs
    logging_steps=100,                   # Registrar cada 100 pasos
    load_best_model_at_end=True,         # Cargar el mejor modelo al final
    metric_for_best_model="accuracy",   # Métrica para seleccionar el mejor modelo
    greater_is_better=True,              # Indicar si mayor valor es mejor
    learning_rate=5e-5,                  # Tasa de aprendizaje
    weight_decay=0.01,                   # Decaimiento del peso
    fp16=True,                           # Acelerar con half precision si está disponible
)

# Función para calcular métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, np.ndarray):
        logits = torch.tensor(logits)
    predictions = torch.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions.numpy())}

# Configurar Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:

# Buscar último checkpoint
checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
last_checkpoint = None
if os.path.exists(checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint-")]
    if checkpoints:
        last_checkpoint = os.path.join(checkpoint_dir, sorted(checkpoints)[-1])
        logging.info(f"Resumiendo entrenamiento desde checkpoint: {last_checkpoint}")

# Entrenar modelo
if last_checkpoint:
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    trainer.train()

# Guardar modelo entrenado
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
logging.info(f"Modelo y tokenizador guardados en {OUTPUT_DIR}")
